In [0]:
"""
Notebook de consolidation et nettoyage des fichiers de simulation sans défaut.

Ce notebook effectue la consolidation de fichiers provenant de différentes sources,
assure une nomenclature cohérente et élimine les doublons pour créer un dataset
propre de simulations sans défaut. Il effectue les opérations suivantes :

1. **Déplacement et consolidation des fichiers** :
   - Déplacement initial de Sans_Defaut2/ vers Reparer/ avec shutil.move
   - Création du dossier de destination final : Dossier_Sans_Defaut_Final/
   - Fonction copier_fichiers() pour copie avec préservation des métadonnées
   - Consolidation depuis 2 sources : Reparer/ et Fichier_Sans_Defaut/
   - Utilisation de shutil.copy2 pour conserver timestamps et permissions
   - Rapport détaillé de chaque fichier copié avec statut [OK]

2. **Mise en place d'une nouvelle numérotation cohérente** :
   - Pattern regex : SimSansIRB4000_(\d+)_ pour extraction des numéros
   - Détection du numéro maximum existant dans le dataset
   - Gestion intelligente des doublons avec dictionnaire seen_numbers
   - Attribution séquentielle pour les premiers occurrences (1, 2, 3...)
   - Attribution au-delà du max pour les doublons détectés
   - Renommage en format uniforme : Sim_X_ (suppression de SansIRB4000)
   - Mapping complet ancien → nouveau numéro pour traçabilité

3. **Vérification de la continuité de la numérotation** :
   - Pattern regex simplifié : Sim_(\d+)_ pour validation
   - Extraction et tri de tous les numéros présents
   - Calcul de la plage attendue : range(1, max+1)
   - Identification explicite des numéros manquants
   - Statistiques : total fichiers, min/max, liste des trous
   - Validation [OK] si séquence continue, [ATTENTION] sinon

4. **Détection des doublons par paramètres physiques** :
   - Pattern regex complexe : N_([\d\.]+)_TOT_([\d\.]+)_Tamb_([\d\.]+)_pamb_([\d\.]+)_
   - Extraction de 4 paramètres : vitesse N, température TOT, Tamb, pamb
   - Création de tuples uniques (N, TOT, Tamb, pamb) comme clés
   - Utilisation d'un set pour détecter les combinaisons dupliquées
   - Rapport détaillé : nombre de combinaisons uniques vs doublons
   - Liste complète des fichiers en doublon avec leurs paramètres

5. **Identification et suppression des fichiers Copy** :
   - Recherche des fichiers contenant " - Copy" dans le nom
   - Première passe : comptage et listing pour vérification
   - Affichage de la liste complète avant suppression
   - Suppression sécurisée avec try/except pour gestion erreurs
   - Rapport de suppression fichier par fichier
   - Confirmation finale du nettoyage effectué

Chemins utilisés :
- Source 1 : /Volumes/dbe_dbx_internships/justin/predictive_maintenance/Sans_Defaut2/
- Source 2 : /Volumes/dbe_dbx_internships/justin/predictive_maintenance/Fichier_Sans_Defaut/
- Temporaire : /Volumes/dbe_dbx_internships/justin/predictive_maintenance/Reparer/
- Final : /Volumes/dbe_dbx_internships/justin/predictive_maintenance/Dossier_Sans_Defaut_Final/

Patterns de nomenclature :
- Entrée : SimSansIRB4000_XXX_N_YYY_TOT_ZZZ_Tamb_AAA_pamb_BBB_*.csv
- Sortie : Sim_X_N_YYY_TOT_ZZZ_Tamb_AAA_pamb_BBB_*.csv
- Paramètres clés : N (vitesse), TOT (température), Tamb, pamb

Résultats obtenus :
- Dataset consolidé dans un dossier unique
- Numérotation séquentielle continue (1 → N)
- Élimination des doublons de type " - Copy"
- Validation de l'unicité des combinaisons de paramètres

"""

1. Déplacement d'un dossier de défauts vers un dossier de destination final

In [0]:
import os
import shutil

source_folder = "/Volumes/dbe_dbx_internships/justin/predictive_maintenance/Sans_Defaut2/"
destination_folder = "/Volumes/dbe_dbx_internships/justin/predictive_maintenance/Reparer/"

os.makedirs(destination_folder, exist_ok=True)

for filename in os.listdir(source_folder):
    if filename.endswith(".csv"):
        src_path = os.path.join(source_folder, filename)
        dest_path = os.path.join(destination_folder, filename)

        shutil.move(src_path, dest_path)
        print(f"[OK] Déplacé : {filename}")


[OK] Déplacé : SimSansIRB4000_100_N_65.7144_TOT_671.3824_Tamb_301.2526_pamb_0.96_percAsh_0.0001207_Foul_0_SF_0.csv
[OK] Déplacé : SimSansIRB4000_101_N_78.2189_TOT_529.4493_Tamb_277.8296_pamb_1.02_percAsh_0.0001055_Foul_0_SF_0.csv
[OK] Déplacé : SimSansIRB4000_102_N_75.7691_TOT_538.6972_Tamb_264.7368_pamb_0.94_percAsh_0.0001498_Foul_0_SF_0.csv
[OK] Déplacé : SimSansIRB4000_103_N_71.8571_TOT_663.1006_Tamb_307.7756_pamb_1.01_percAsh_0.0001344_Foul_0_SF_0.csv
[OK] Déplacé : SimSansIRB4000_104_N_61.5342_TOT_669.6971_Tamb_268.8743_pamb_1.00_percAsh_0.0001062_Foul_0_SF_0.csv
[OK] Déplacé : SimSansIRB4000_105_N_94.6868_TOT_541.8078_Tamb_299.5880_pamb_0.97_percAsh_0.0001667_Foul_0_SF_0.csv
[OK] Déplacé : SimSansIRB4000_106_N_68.0295_TOT_631.8241_Tamb_300.1053_pamb_1.03_percAsh_0.0000928_Foul_0_SF_0.csv
[OK] Déplacé : SimSansIRB4000_107_N_86.0384_TOT_507.8701_Tamb_269.9311_pamb_1.01_percAsh_0.0001632_Foul_0_SF_0.csv
[OK] Déplacé : SimSansIRB4000_108_N_69.8473_TOT_621.9459_Tamb_312.3757_pamb_0.98

In [0]:
folder_reparer = "/Volumes/dbe_dbx_internships/justin/predictive_maintenance/Reparer/"
folder_sans_defaut2 = "/Volumes/dbe_dbx_internships/justin/predictive_maintenance/Fichier_Sans_Defaut/"

folder_final = "/Volumes/dbe_dbx_internships/justin/predictive_maintenance/Dossier_Sans_Defaut_Final/"

os.makedirs(folder_final, exist_ok=True)

def copier_fichiers(source_folder, destination_folder): # Fonction pour copier les fichiers .csv d'un dossier source vers destination
    for filename in os.listdir(source_folder):
        if filename.endswith(".csv"):
            src_path = os.path.join(source_folder, filename)
            dest_path = os.path.join(destination_folder, filename)
            shutil.copy2(src_path, dest_path)
            print(f"[OK] Copié : {filename}")

print("\n=== Copie depuis REPARER ===")
copier_fichiers(folder_reparer, folder_final)

print("\n=== Copie depuis SANS_DEFAUT2 ===")
copier_fichiers(folder_sans_defaut2, folder_final)

print("\n=== OPÉRATION TERMINÉE ===")
print(f"Tous les fichiers sont maintenant dans : {folder_final}")



=== Copie depuis REPARER ===
[OK] Copié : SimSansIRB4000_1000_N_63.9245_TOT_682.7964_Tamb_302.3835_pamb_0.96_percAsh_0.0001664_Foul_0_SF_0.csv
[OK] Copié : SimSansIRB4000_1001_N_70.9367_TOT_510.7544_Tamb_270.5715_pamb_0.99_percAsh_0.0000956_Foul_0_SF_0.csv
[OK] Copié : SimSansIRB4000_1002_N_91.1267_TOT_514.2988_Tamb_296.2173_pamb_0.95_percAsh_0.0001384_Foul_0_SF_0.csv
[OK] Copié : SimSansIRB4000_1003_N_65.1120_TOT_539.5990_Tamb_307.5383_pamb_0.91_percAsh_0.0001369_Foul_0_SF_0.csv
[OK] Copié : SimSansIRB4000_1004_N_64.1413_TOT_579.8318_Tamb_290.0356_pamb_0.95_percAsh_0.0001631_Foul_0_SF_0.csv
[OK] Copié : SimSansIRB4000_1005_N_81.6682_TOT_536.5655_Tamb_280.2251_pamb_1.03_percAsh_0.0001329_Foul_0_SF_0.csv
[OK] Copié : SimSansIRB4000_1006_N_89.9884_TOT_527.9482_Tamb_263.3799_pamb_0.97_percAsh_0.0001085_Foul_0_SF_0.csv
[OK] Copié : SimSansIRB4000_1007_N_86.0299_TOT_673.6318_Tamb_280.5389_pamb_0.94_percAsh_0.0000911_Foul_0_SF_0.csv
[OK] Copié : SimSansIRB4000_1008_N_94.3130_TOT_530.2204_Ta

2. Mise en place d'une nouvelle numérotation dans le dossier de destination final

In [0]:
import re

folder_final = "/Volumes/dbe_dbx_internships/justin/predictive_maintenance/Dossier_Sans_Defaut_Final/"

pattern = re.compile(r"SimSansIRB4000_(\d+)_") # Expression régulière par Regex pour extraire les numéros après SimSansIRB4000_XXX_

fichiers_csv = [f for f in os.listdir(folder_final) if f.endswith(".csv")]

seen_numbers = set() # Dictionnaire pour vérifier les doublons

all_numbers = []
for filename in fichiers_csv:
    match = pattern.search(filename)
    if match:
        num = int(match.group(1))
        all_numbers.append(num)

if all_numbers:
    current_max = max(all_numbers) # Trouver le numéro max existant
else:
    current_max = 0

print(f"[INFO] Numéro max existant : {current_max}")

new_number = 1 # Nouvelle numérotation
number_mapping = {}  # Pour traquer les remplacements

for filename in sorted(fichiers_csv):
    src_path = os.path.join(folder_final, filename)

    match = pattern.search(filename)
    if match:
        original_num = int(match.group(1))

        if original_num not in seen_numbers: # Vérifier si ce numéro a déjà été vu
            new_num = new_number
            seen_numbers.add(original_num)
            new_number += 1  # Premier passage → on le garde et on incrémente new_number !!
        else:
            current_max += 1 # Doublon → on attribue un nouveau numéro > current_max
            new_num = current_max

        number_mapping[filename] = new_num

        new_filename = re.sub(r"SimSansIRB4000_\d+_", f"Sim_{new_num}_", filename) # Construire le nouveau nom
        dest_path = os.path.join(folder_final, new_filename)

        os.rename(src_path, dest_path)
        print(f"[OK] {filename} → {new_filename}")

    else:
        print(f"[SKIP] Fichier ne correspondant pas au pattern : {filename}")

print("\n=== OPÉRATION TERMINÉE ===")


[INFO] Numéro max existant : 1699
[OK] SimSansIRB4000_1000_N_63.9245_TOT_682.7964_Tamb_302.3835_pamb_0.96_percAsh_0.0001664_Foul_0_SF_0.csv → Sim_1_N_63.9245_TOT_682.7964_Tamb_302.3835_pamb_0.96_percAsh_0.0001664_Foul_0_SF_0.csv
[OK] SimSansIRB4000_1000_N_96.4651_TOT_621.2264_Tamb_287.4173_pamb_1.03_percAsh_0.0001315_Foul_0_SF_0.csv → Sim_1700_N_96.4651_TOT_621.2264_Tamb_287.4173_pamb_1.03_percAsh_0.0001315_Foul_0_SF_0.csv
[OK] SimSansIRB4000_1001_N_70.9367_TOT_510.7544_Tamb_270.5715_pamb_0.99_percAsh_0.0000956_Foul_0_SF_0.csv → Sim_2_N_70.9367_TOT_510.7544_Tamb_270.5715_pamb_0.99_percAsh_0.0000956_Foul_0_SF_0.csv
[OK] SimSansIRB4000_1001_N_99.3150_TOT_698.4021_Tamb_307.9579_pamb_0.99_percAsh_0.0001487_Foul_0_SF_0.csv → Sim_1701_N_99.3150_TOT_698.4021_Tamb_307.9579_pamb_0.99_percAsh_0.0001487_Foul_0_SF_0.csv
[OK] SimSansIRB4000_1002_N_75.9853_TOT_543.5782_Tamb_307.2315_pamb_1.03_percAsh_0.0000985_Foul_0_SF_0.csv → Sim_3_N_75.9853_TOT_543.5782_Tamb_307.2315_pamb_1.03_percAsh_0.0000985_F

3. Vérification de la continuité des numéros de fichiers après procédure

In [0]:
folder_final = "/Volumes/dbe_dbx_internships/justin/predictive_maintenance/Dossier_Sans_Defaut_Final/"

pattern = re.compile(r"Sim_(\d+)_") #Expression régulière Regex pour extraire les numéros

fichiers_csv = [f for f in os.listdir(folder_final) if f.endswith(".csv")]

num_list = []

for filename in fichiers_csv:
    match = pattern.search(filename)
    if match:
        num = int(match.group(1))
        num_list.append(num)
    else:
        print(f"[SKIP] Fichier non conforme : {filename}")

if not num_list:
    print("[ERREUR] Aucun numéro trouvé !")
else:
    num_list.sort()
    min_num = num_list[0]
    max_num = num_list[-1]
    expected_range = list(range(1, max_num + 1))

    missing = [n for n in expected_range if n not in num_list]

    print(f"\n[INFO] Total fichiers : {len(num_list)}")
    print(f"[INFO] Numéros min → max : {min_num} → {max_num}")

    if missing:
        print(f"[ATTENTION] Numéros manquants ({len(missing)}) : {missing}")
    else:
        print("[OK] Tous les fichiers sont bien en séquence continue !")



[INFO] Total fichiers : 3291
[INFO] Numéros min → max : 1 → 3291
[OK] Tous les fichiers sont bien en séquence continue !


4. Identification des doublons avec les paramètres ambiants et rentrés en consigne

In [0]:
folder_final = "/Volumes/dbe_dbx_internships/justin/predictive_maintenance/Dossier_Sans_Defaut_Final/"

pattern = re.compile( # Expression régulière Regex  pour extraire N, TOT, Tamb, pamb
    r"N_([\d\.]+)_TOT_([\d\.]+)_Tamb_([\d\.]+)_pamb_([\d\.]+)_"
)

fichiers_csv = [f for f in os.listdir(folder_final) if f.endswith(".csv")]

combinaisons = set()
doublons = []

for filename in fichiers_csv:
    match = pattern.search(filename)
    if match:
        n_val = float(match.group(1))
        tot_val = float(match.group(2))
        tamb_val = float(match.group(3))
        pamb_val = float(match.group(4))

        combinaison = (n_val, tot_val, tamb_val, pamb_val)

        if combinaison in combinaisons:
            doublons.append((filename, combinaison))
        else:
            combinaisons.add(combinaison)
    else:
        print(f"[SKIP] Fichier non conforme : {filename}")

print(f"\n[INFO] Nombre total de fichiers : {len(fichiers_csv)}")
print(f"[INFO] Nombre de combinaisons uniques trouvées : {len(combinaisons)}")

if doublons:
    print(f"\n[ATTENTION] Doublons détectés ! ({len(doublons)}) :")
    for d in doublons:
        print(f"  → {d[0]} | Combinaison : {d[1]}")
else:
    print("\n[OK] Toutes les combinaisons (N, TOT, Tamb, pamb) sont uniques ! ")



[INFO] Nombre total de fichiers : 3291
[INFO] Nombre de combinaisons uniques trouvées : 3289

[ATTENTION] Doublons détectés ! (2) :
  → Sim_2578_N_98.4556_TOT_515.9834_Tamb_282.4457_pamb_0.98_percAsh_0.0001355_Foul_0_SF_0.csv | Combinaison : (98.4556, 515.9834, 282.4457, 0.98)
  → Sim_905_N_75.2134_TOT_574.4167_Tamb_284.3637_pamb_0.95_percAsh_0.0001401_Foul_0_SF_0 - Copy.csv | Combinaison : (75.2134, 574.4167, 284.3637, 0.95)


5. Identification des fichiers Copy et suppression 

In [0]:
folder_final = "/Volumes/dbe_dbx_internships/justin/predictive_maintenance/Dossier_Sans_Defaut_Final/"

copy_files = [f for f in os.listdir(folder_final) if f.endswith(".csv") and " - Copy" in f] # Chercher tous les fichiers contenant '- Copy' avant l'extension .csv

print(f"\n[INFO] Nombre de fichiers '- Copy.csv' dans le dossier : {len(copy_files)}")

if copy_files:
    print("\nListe des fichiers 'Copy' :")
    for f in copy_files:
        print(f" → {f}")
else:
    print("[OK] Aucun fichier 'Copy' détecté.")



[INFO] Nombre de fichiers '- Copy.csv' dans le dossier : 5

Liste des fichiers 'Copy' :
 → Sim_2577_N_98.4556_TOT_515.9834_Tamb_282.4457_pamb_0.98_percAsh_0.0001355_Foul_0_SF_0 - Copy.csv
 → Sim_2579_N_83.5922_TOT_572.0433_Tamb_264.4217_pamb_0.92_percAsh_0.0001340_Foul_0_SF_0 - Copy.csv
 → Sim_2580_N_87.3016_TOT_506.9938_Tamb_285.1599_pamb_1.00_percAsh_0.0001700_Foul_0_SF_0 - Copy.csv
 → Sim_2581_N_95.4326_TOT_509.9443_Tamb_279.7328_pamb_0.97_percAsh_0.0001337_Foul_0_SF_0 - Copy.csv
 → Sim_905_N_75.2134_TOT_574.4167_Tamb_284.3637_pamb_0.95_percAsh_0.0001401_Foul_0_SF_0 - Copy.csv


In [0]:
folder_final = "/Volumes/dbe_dbx_internships/justin/predictive_maintenance/Dossier_Sans_Defaut_Final/"

copy_files = [f for f in os.listdir(folder_final) if f.endswith(".csv") and " - Copy" in f]

if copy_files:
    print(f"\n[INFO] Suppression des fichiers '- Copy.csv' ({len(copy_files)}) :")
    for f in copy_files:
        file_path = os.path.join(folder_final, f)
        try:
            os.remove(file_path)
            print(f"[SUPPRIMÉ] {f}")
        except Exception as e:
            print(f"[ERREUR] Impossible de supprimer {f} : {e}")
    print("\n[OK] Nettoyage terminé ")
else:
    print("[OK] Aucun fichier '- Copy.csv' à supprimer. ")



[INFO] Suppression des fichiers '- Copy.csv' (5) :
[SUPPRIMÉ] Sim_2577_N_98.4556_TOT_515.9834_Tamb_282.4457_pamb_0.98_percAsh_0.0001355_Foul_0_SF_0 - Copy.csv
[SUPPRIMÉ] Sim_2579_N_83.5922_TOT_572.0433_Tamb_264.4217_pamb_0.92_percAsh_0.0001340_Foul_0_SF_0 - Copy.csv
[SUPPRIMÉ] Sim_2580_N_87.3016_TOT_506.9938_Tamb_285.1599_pamb_1.00_percAsh_0.0001700_Foul_0_SF_0 - Copy.csv
[SUPPRIMÉ] Sim_2581_N_95.4326_TOT_509.9443_Tamb_279.7328_pamb_0.97_percAsh_0.0001337_Foul_0_SF_0 - Copy.csv
[SUPPRIMÉ] Sim_905_N_75.2134_TOT_574.4167_Tamb_284.3637_pamb_0.95_percAsh_0.0001401_Foul_0_SF_0 - Copy.csv

[OK] Nettoyage terminé 
